In [ ]:
import os
import sys

from dotenv import load_dotenv

pkg_path = os.path.abspath("../")
if pkg_path not in sys.path:
    sys.path.append(pkg_path)

load_dotenv()

## Criação do agente


Criação do agente especialista em filmes como checkpoints salvos na memória de trabalho (`MemorySaver`).

A orquestração das ações se baseia no modelo `gpt-4o` da OpenAI.

In [ ]:
from langchain_openai import AzureChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from src.agent import ReactAgent

llm_agent = AzureChatOpenAI(model="gpt-4o", temperature=0)
agent = ReactAgent(llm=llm_agent, memory=MemorySaver())

## Protótipo de interface - Gradio

[Gradio](https://www.gradio.app) é um pacote Python de código aberto que permite a criação de aplicativos web para modelos de machine learning, API ou função Python. Além disso, é pssível compartilhar a aplicação criada por meio de um link público.

In [ ]:
!pip install gradio -q

In [ ]:
# Front end web app for the assistant
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []
    
    def user(user_message, history):
        answer = agent.answer_question(question=user_message,  thread_id="1")
        history.append((user_message, answer["response"]))
       
        return gr.update(value=""), history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)

## Requsição assíncrona à API em FastAPI

In [ ]:
import httpx

async def make_request(query: str, session_id: str):
    url = 'http://localhost:8000/pushseer'
    params = {
        'query': query,
        'session_id': session_id
    }
    headers = {
        'accept': 'application/json',
        'x-api-key': os.getenv('ASSISTANT_API_KEY')
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params, headers=headers)
        return response.json()


In [ ]:
answer = await make_request(query="Atores de Matrix", session_id="1")
print(answer)